In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd

In [2]:
import sys

In [3]:
sys.path.append('../')

In [4]:
dat_prefix = '../../data/dats'
inp_prefix = '../../data/inps'

leaf = {
    '3v363': '03_V363_Rotor',
    'v769': '04_V769_Rotor',
    '5v363': '05_V363_Rotor_dsg2',
    'c346rs': 'C346RS_frnt_rotor_modal_separation_10Jun25',
    'v801': 'V801_frnt_rotor_modal_separation_17Jun25'
}

def dat(key: str) -> str:
    return dat_prefix + '/' + leaf[key] + '.dat'

def inp(key: str) -> str:
    return inp_prefix + '/' + leaf[key] + '.inp'

def both(key: str):
    return (dat(key), inp(key))

In [5]:
from analyser.modalAnalyser import ModalAnalyser
from parser.datParser import DATParser
from parser.inpParser import INPParser
from parser.modalParser import ModalParser

In [9]:
model = ModalParser(*both('v801'))
analyser = ModalAnalyser(model)

In [133]:
analyser.check()

In-plane modes: [43, 44, 64, 65, 69, 77]
Near in-plane modes: [42, 45, 70, 71, 74, 75, 76, 78, 79]
42 oop: 65.62157524965924 passed
45 oop: 44.589447933072705 passed
70 oop: 94.12572741361068 passed
71 oop: 94.12432848459933 passed
74 oop: 6.426250862960288 passed
75 oop: 15.735282020072367 passed
76 oop: 54.55761527431685 passed
78 oop: 55.322563712589336 passed
79 oop: 5.147278561957867 passed


True

In [6]:
model = ModalParser(*both('c346rs'))
analyser = ModalAnalyser(model)

In [7]:
for i in range(1, 132):
    if analyser.is_tangential(i):
        print(i, analyser.is_rigid_rotation(i, rigid_ratio_thres=1e-1, return_ratio=True))

6 (np.True_, np.float64(0.9999981880348765))
46 (np.False_, np.float64(1.8197611471582153e-08))
47 (np.False_, np.float64(2.6555849929082205e-07))
68 (np.False_, np.float64(5.97488960221783e-06))
69 (np.False_, np.float64(4.116589194577371e-06))
73 (np.False_, np.float64(2.538314482357084e-05))
74 (np.False_, np.float64(0.0002952412021300223))
75 (np.True_, np.float64(0.9467315341113544))
100 (np.False_, np.float64(2.5040155499186403e-07))
101 (np.False_, np.float64(6.279193870752669e-07))
129 (np.True_, np.float64(0.21251401977273646))


In [8]:
analyser.check(tangential=True)

In-plane modes: [46, 47, 68, 69, 73, 74, 100, 101]
Near in-plane modes: [64, 65, 66, 67, 70, 71, 72, 75, 76, 77, 99]
64 oop: 95.83343048951669 passed
failed here
65 oop: 95.83343109838387 passed
failed here
66 oop: 40.08328606500776 passed
67 oop: 40.08188844770941 passed
70 oop: 0.9691565074584988 passed
71 oop: 26.90025349962847 passed
72 oop: 26.899207109899407 passed
75 oop: 0.5360215702609913 passed
76 oop: 22.429000926348476 passed
77 oop: 23.330070876072128 passed
99 oop: 80.32828492320894 passed


True